In [1]:
from pymmcore_plus import CMMCorePlus
from fov import FOV
import useq
from useq import MDAEvent
from queue import Queue
import numpy as np
from matplotlib import pyplot as plt
from napari import Viewer
import numpy as np
from queue import Queue
from pymmcore_plus import CMMCorePlus
from useq import MDAEvent
import useq
from useq._channel import Channel
from MDAEngine_DMD import MDAEngine_DMD
from controller import Controller, Analyzer
import pandas as pd
import random
from utils import ImgType, MetadataDict
from stimulation import StimExtraParameters
from dmd import DMD
from hardware import load_config
import os
import tempfile
from datetime import datetime
import pymmcore
import napari
from tracking import TrackerTrackpy
from utils import create_folders

In [2]:
#TODO check if dmd calibration works
#TODO adjust napari live view, check how they change brightness in napari-umanager (or directly use napari-umanager?)
#TODO add napari widget to store/display positions
#TODO generate necessary folders beforehand
#TODO add to known issues: conda install -c conda-forge llvm-openmp if there are issues with threads on M1

## Load uManager and set default devices

In [3]:
mmc = CMMCorePlus() #Create a new instance of the CMMCorePlus class
mmc.loadSystemConfiguration('local/local_config.cfg')  #Load the system configuration file (standard uManager .cfg file)
#load_config(mmc) #Set the default device from the YAML file (specific to this package)

In [4]:
# Create a new log file in the temp directory
log_dir = tempfile.gettempdir()
log_dir = 'Z:\\lhinder\\data\\rtm_mm_data\\exp_355\\'
log_file = os.path.join(log_dir, "log.txt")

# Get the current date and time
current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")


print(f"Logfile: {log_file}")

# Open the log file in append mode and write the date and time
with open(log_file, "w") as file: #w: write (delete old content), a: append
    file.write(f"{current_time}\n")
    file.write(f"User ID: {mmc.getUserId()}")

    file.write(f"PyMMCore version {pymmcore.__version__}\n")
    file.write(f"{mmc.getAPIVersionInfo()}")
    file.write(f"{mmc.getVersionInfo()}\n\n")

    file.write("Loaded devices:\n")
    file.write("\n".join(str(device) for device in mmc.getLoadedDevices()))
    file.write("\n\n")

Logfile: Z:\lhinder\data\rtm_mm_data\exp_355\log.txt


In [5]:
#viewer = napari.Viewer()
#image_layer = viewer.add_image(np.ones((1,1)))

In [6]:
path = log_dir
create_folders(path,['stim','raw','mask','stim_mask'])

Directory Z:\lhinder\data\rtm_mm_data\exp_355\stim already exists
Directory Z:\lhinder\data\rtm_mm_data\exp_355\raw already exists
Directory Z:\lhinder\data\rtm_mm_data\exp_355\mask already exists
Directory Z:\lhinder\data\rtm_mm_data\exp_355\stim_mask already exists


## Create list with FOVs

In [20]:
# get current position
pos = (mmc.getXPosition(),mmc.getYPosition())
pos_list = [pos]

In [21]:
print(mmc.getXPosition())
print(mmc.getYPosition())

16539.000246450305
-12507.900186382234


In [22]:
#pos_list = [(2,2)]
fovs = []

for i,pos in enumerate(pos_list):
    print(pos)
    fov = FOV(pos=pos,
              index =i,
              path=path,
              metadata={},
              properties={'stim_property': 'top'},
              )
    fovs.append(fov)



(16539.000246450305, -12507.900186382234)


## Create a DF with all planned acquisitions and stimulations

In [23]:
df_acquire = pd.DataFrame(columns=['fov', 'timestep', 'time','time_experiment', 'treatment', 'acquired','stim', 'channels', 'channel_stim'])

time_between_frames = 5 #time in seconds between frames
stim_timesteps = [2,3]  # list of timesteps
stim_timesteps= []  # list of timesteps

timestep = range(2)  # 0-20
treatments = ['stim_top', 'stim_mid',]  # list of treatments
fovs:list[FOV] = fovs
channels = [['miRFP','mCherry']]
channels_exposure = [[200,100]]
channel_stim = ['mCitrine']
channel_stim_exposure = [100]

ImgType.IMG_RAW

# Loop over the FOVs and randomly assign one of the treatments to it
treatments_shuffled = treatments.copy()
random.shuffle(treatments_shuffled)
for fov in fovs:
    treatment = treatments_shuffled[fov.index % len(treatments_shuffled)]
    for timestep in timestep:
        new_row = { 'fov_object': fov,
                    'fov':fov.index,
                    'timestep': timestep,
                    'time': timestep*time_between_frames,
                    'treatment': treatment,
                    'acquired': False,
                    'stim': False,
                    'channels': channels,
                    'channels_exposure':channels_exposure,
                    'channel_stim' : channel_stim,
                    'channel_stim_exposure' : channel_stim_exposure,
                    'fname' : f'{str(fov.index).zfill(3)}_{str(timestep).zfill(5)}',
                    }
        df_acquire = pd.concat([df_acquire, pd.DataFrame(new_row, index=[0])])

for timestep in stim_timesteps:
    df_acquire.loc[df_acquire['timestep'] == timestep, 'stim'] = True
df_acquire = df_acquire.sort_values(by=['timestep', 'fov'])

df_acquire

,fov,timestep,time,time_experiment,treatment,acquired,stim,channels,channel_stim,fov_object,channels_exposure,channel_stim_exposure,fname
0,0,0,0,NaN,stim_mid,False,False,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x00000206ECA062B0>,"[200, 100]",100.0,000_00000
0,0,1,5,NaN,stim_mid,False,False,"[miRFP, mCherry]",mCitrine,<fov.FOV object at 0x00000206ECA062B0>,"[200, 100]",100.0,000_00001


## Take an image at the current location

In [24]:
print(f'Channels to acquire: {channels[0]}')

Channels to acquire: ['miRFP', 'mCherry']


## Run on virtual system 

In [25]:
from add_frame import ImageProcessingPipeline
from segmentation import SegmentatorStardist
from pymmcore_plus.mda import MDAEngine
from stimulation import StimExtraParameters
from controller import Analyzer

segmentator = SegmentatorStardist('2D_versatile_fluo')
stimulator = StimExtraParameters()
tracker = TrackerTrackpy()
pipeline = ImageProcessingPipeline(segmentator,stimulator,tracker)
analyzer = Analyzer(pipeline)


Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [ ]:
#mmc = CMMCorePlus()

#mmc.loadSystemConfiguration()

#setup camera
#mmc.initializeDevice("Camera");
#mmc.setCameraDevice("Camera");

# Register the custom engine with the runner
dmd = DMD(mmc, test_mode = False)
mda_engine_dmd = MDAEngine_DMD(dmd)

mmc.mda.set_engine(mda_engine_dmd)

queue = Queue()
controller = Controller(analyzer, mmc, queue)

# Start the acquisition
controller.run(df_acquire)
#controller.run_test()


2023-09-22 15:23:33,744 - pymmcore-plus - INFO - (_runner.py:216) MDA Started: GeneratorMDASequence()


Current timestep: 0
Current timestep: 1


2023-09-22 15:23:33,750 - pymmcore-plus - INFO - (_runner.py:184) channel=Channel(config='miRFP') exposure=200.0 min_start_time=nan x_pos=16539.000246450305 y_pos=-12507.900186382234 metadata={'fov': <fov.FOV object at 0x00000206ECA062B0>, 'img_type': <ImgType.IMG_RAW: 1>, 'last_channel': False, 'timestep': 0, 'time': 0, 'time_experiment': nan, 'fname': '000_00000', 'stim': False, 'channels': ['miRFP', 'mCherry'], 'channel': 'miRFP'}


No DMD mask found
16539.000246450305 -12507.900186382234


2023-09-22 15:23:34,787 - pymmcore-plus - INFO - (_runner.py:184) channel=Channel(config='mCherry') exposure=100.0 min_start_time=nan x_pos=16539.000246450305 y_pos=-12507.900186382234 metadata={'fov': <fov.FOV object at 0x00000206ECA062B0>, 'img_type': <ImgType.IMG_RAW: 1>, 'last_channel': True, 'timestep': 0, 'time': 0, 'time_experiment': nan, 'fname': '000_00000', 'stim': False, 'channels': ['miRFP', 'mCherry'], 'channel': 'mCherry'}


...

2023-09-22 15:23:37,071 - pymmcore-plus - INFO - (_runner.py:184) channel=Channel(config='miRFP') exposure=200.0 min_start_time=nan x_pos=16539.000246450305 y_pos=-12507.900186382234 metadata={'fov': <fov.FOV object at 0x00000206ECA062B0>, 'img_type': <ImgType.IMG_RAW: 1>, 'last_channel': False, 'timestep': 1, 'time': 5, 'time_experiment': nan, 'fname': '000_00001', 'stim': False, 'channels': ['miRFP', 'mCherry'], 'channel': 'miRFP'}


No DMD mask found
16539.000246450305 -12507.900186382234


2023-09-22 15:23:39,344 - pymmcore-plus - INFO - (_runner.py:184) channel=Channel(config='mCherry') exposure=100.0 min_start_time=nan x_pos=16539.000246450305 y_pos=-12507.900186382234 metadata={'fov': <fov.FOV object at 0x00000206ECA062B0>, 'img_type': <ImgType.IMG_RAW: 1>, 'last_channel': True, 'timestep': 1, 'time': 5, 'time_experiment': nan, 'fname': '000_00001', 'stim': False, 'channels': ['miRFP', 'mCherry'], 'channel': 'mCherry'}


No DMD mask found
16539.000246450305 -12507.900186382234


2023-09-22 15:23:41,592 - pymmcore-plus - INFO - (_runner.py:308) MDA Finished: GeneratorMDASequence()


In [15]:
configs = mmc.getAvailableConfigs('Channel')
for config in configs:
    print(config)


GFP
mCherry
mCitrine
miRFP


In [6]:
assert np.array_equal(segmentator.segment(np.zeros((100,100))), np.zeros((100,100)))
assert np.array_equal(stimulator.get_stim_mask(np.zeros((10,10)),FOV(0,0,'/tmp/',{},{'stim_property': 'top'})),np.ones((10,10)))

(100, 100)


In [8]:
from add_frame import store_img
fov = FOV(pos=(0,0),index =i, path='/tmp/', metadata={'stim_property': 'top'},properties={},)
metadata = {'fov':0,'img_type':ImgType.IMG_RAW,'fname':'test', 'fov_object':fov}
store_img(np.ones((4,10,10)),metadata,'/tmp/')

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/Users/luhin/miniforge3/envs/micro-llm/lib/python3.9/threading.py", line 950, in _bootstrap_inner
    self.run()
  File "/Users/luhin/miniforge3/envs/micro-llm/lib/python3.9/threading.py", line 888, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/luhin/sync/phd/code/rtm/add_frame.py", line 41, in run
    stim_mask = self.stimulator.get_stim_mask(segmented)
TypeError: get_stim_mask() missing 1 required positional argument: 'fov'


: 

: 

In [ ]:
mmc.snapImage()
mmc.getImage()
mmc.setExposure(100)
mmc.getCameraDevice()

'Camera'

## Run on system with DMD

In [ ]:
from add_frame import ImageProcessingPipeline
from segmentation import SegmentatorStardist

segmentator = SegmentatorStardist('2D_versatile_fluo')
stimulator = StimExtraParameters()
pipeline = ImageProcessingPipeline(segmentator,stimulator)
analyzer = Analyzer(pipeline)



mmc = CMMCorePlus()
dmd = DMD(mmc)

mmc.loadSystemConfiguration()
mmc.mda.set_engine(MDAEngine_DMD(mmc,dmd))

# Apply the custom acquisition engine
# Register the custom engine with the runner
core = CMMCorePlus.instance()
#core.mda.set_engine(MDAEngine_DMD(core))

# create the Queue that will hold the MDAEvents
queue = Queue()



controller = Controller(analyzer, mmc , queue, dmd)
# Start the acquisition
controller.run()

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


RuntimeError: No device with label ""

In [ ]:
"""Simple simulator demonstrating event-driven acquisitions with pymmcore-plus."""
import random
import time
from queue import Queue

import numpy as np
from pymmcore_plus import CMMCorePlus
from useq import MDAEvent, MDASequence


class Analyzer:
    """Analyzes images and returns a dict of results."""

    def run(self, data) -> dict:
        pass
        # Fake analysis; randomly return a dict with a value of None 10% of the time
        #if random.random() < 0.1:
        #return {"result": "STOP"}

       # else:
        #return {"result": random.random()}


class Controller:
    STOP_EVENT = object()

    def __init__(self, analyzer: Analyzer, mmc: CMMCorePlus, queue: Queue):
        self._analyzer = analyzer  # analyzer of images
        self._queue = queue  # queue of MDAEvents
        self._results: dict = {}  # results of analysis

        self._mmc = mmc
        mmc.mda.events.frameReady.connect(self._on_frame_ready)

    def _on_frame_ready(self, img: np.ndarray, event: MDAEvent) -> None:
        # Analyze the image
        #print(event)
        self._results = self._analyzer.run(img)
        print(event.metadata['fov'])

    def run(self) -> None:
        # convert the queue to an iterable
        queue_sequence = iter(self._queue.get, self.STOP_EVENT)

        # Start the acquisition (run_mda is non-blocking)
        self._mmc.run_mda(queue_sequence)

        # Queue the first image acquisition
## Create MDASequence with a time delay of 1 second
        
        interval = 2
        nb_fovs = 2
        delay = interval/nb_fovs

        mda_sequence_fov_0 = MDASequence(delay=10)
        mda_sequence_fov_1 = MDASequence(exposure = 10)

        channel = {'config': 'Cy5', 'exposure': 10}

        for frame in range(3):
            self._queue.put(MDAEvent(exposure=10,mda_sequence=mda_sequence_fov_0, min_start_time=frame *interval,metadata={'fov':0},channel=channel))
            self._queue.put(MDAEvent(exposure=10,mda_sequence=mda_sequence_fov_1, min_start_time=frame *interval+delay,metadata={'fov':1}))


       # self._queue.put(intitial_event)
        self._queue.put(self.STOP_EVENT)


# Setup the MM Core
mmc = CMMCorePlus()
mmc.loadSystemConfiguration()

# create the Queue that will hold the MDAEvents
q = Queue()

# Setup the controller and analyzer
analyzer = Analyzer()
controller = Controller(analyzer, mmc, q)

# Start the acquisition
controller.run()





2023-08-30 12:55:11.051 | INFO     | pymmcore_plus.mda._runner:_prepare_to_run:221 - MDA Started: GeneratorMDASequence()
2023-08-30 12:55:11.052 | INFO     | pymmcore_plus.mda._runner:_run:186 - channel=Channel(config='Cy5') exposure=10.0 min_start_time=0.0 metadata={'fov': 0}


0


2023-08-30 12:55:12.055 | INFO     | pymmcore_plus.mda._runner:_run:186 - exposure=10.0 min_start_time=1.0 metadata={'fov': 1}


1


2023-08-30 12:55:13.057 | INFO     | pymmcore_plus.mda._runner:_run:186 - channel=Channel(config='Cy5') exposure=10.0 min_start_time=2.0 metadata={'fov': 0}


...

1


2023-08-30 12:55:15.057 | INFO     | pymmcore_plus.mda._runner:_run:186 - channel=Channel(config='Cy5') exposure=10.0 min_start_time=4.0 metadata={'fov': 0}


0


2023-08-30 12:55:16.057 | INFO     | pymmcore_plus.mda._runner:_run:186 - exposure=10.0 min_start_time=5.0 metadata={'fov': 1}
2023-08-30 12:55:16.074 | INFO     | pymmcore_plus.mda._runner:_finish_run:313 - MDA Finished: GeneratorMDASequence()


1


In [10]:
def normalize_mi_ma(x, mi, ma, clip=False, eps=1e-20, dtype=np.float32):
    if dtype is not None:
        x   = x.astype(dtype,copy=False)
        mi  = dtype(mi) if np.isscalar(mi) else mi.astype(dtype,copy=False)
        ma  = dtype(ma) if np.isscalar(ma) else ma.astype(dtype,copy=False)
        eps = dtype(eps)

def normalize(x, pmin=3, pmax=99.8, axis=None, clip=False, eps=1e-20, dtype=np.float32):
    """Percentile-based image normalization."""

    mi = np.percentile(x.flatten(),pmin)
    ma = np.percentile(x.flatten(),pmax)
    return normalize_mi_ma(x, mi, ma, clip=clip, eps=eps, dtype=dtype)

normalize(np.ones((1000,1000)))
print(np.ones((1000,1000)).flatten().shape)

(1000000,)


In [2]:
import random
import time
from queue import Queue

import numpy as np
from pymmcore_plus import CMMCorePlus
from useq import MDAEvent, MDASequence


In [4]:
mda_sequence_2 = MDASequence(exposure = 10)
for event in mda_sequence_2:
    intitial_event = event
    print('here') 

In [2]:
import napari
viewer = napari.Viewer()

In [4]:
import napari_umanager
from napari_umanager import napari_manager
napari_manager(viewer, mmc, q)

ModuleNotFoundError: No module named 'napari_umanager'